<a href="https://colab.research.google.com/github/TerriblePepito/TerriblePepitostest/blob/main/changerlesnombres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import re
import xml.etree.ElementTree as ET
from pathlib import Path

# ---- Paths (edit filenames as needed) ----
input_svg  = Path("/content/drive/MyDrive/test input/Chiffre input.svg")
output_svg = Path("/content/drive/MyDrive/test output/Chiffre input_plus2.svg")

ADD_VALUE = 2  # add +2 to every detected integer

# ---- Helper: safely handle SVG namespaces ----
def get_default_namespace(tag: str):
    # tag looks like "{namespace}svg" if namespaced, else "svg"
    m = re.match(r"\{(.+)\}", tag)
    return m.group(1) if m else None

# ---- Load & parse SVG ----
tree = ET.parse(input_svg)
root = tree.getroot()

ns = get_default_namespace(root.tag)
nsmap = {"svg": ns} if ns else {}

# Regex to match integers like -3, 0, 12
int_re = re.compile(r"^-?\d+$")

changed = 0
visited = 0

def process_text_node(elem):
    """
    Modify elem.text and elem.tail if they are pure integers (e.g. "12", "-3").
    Returns how many changes were made.
    """
    nonlocal_changed = 0

    # elem.text
    if elem.text is not None:
        t = elem.text.strip()
        if int_re.match(t):
            new_val = str(int(t) + ADD_VALUE)
            # preserve surrounding whitespace if any
            elem.text = elem.text.replace(t, new_val)
            nonlocal_changed += 1

    # elem.tail (rarely used for SVG text, but safe)
    if elem.tail is not None:
        t = elem.tail.strip()
        if int_re.match(t):
            new_val = str(int(t) + ADD_VALUE)
            elem.tail = elem.tail.replace(t, new_val)
            nonlocal_changed += 1

    return nonlocal_changed

# ---- Find all text-bearing elements: <text> and <tspan> (and any descendants) ----
# We'll walk the entire tree and edit any node whose .text is a pure integer.
for elem in root.iter():
    visited += 1
    changed += process_text_node(elem)

# ---- Basic validation: did we actually change anything? ----
if changed == 0:
    print("No numeric <text>/<tspan> nodes were found to edit.")
    print("Likely causes:")
    print("  - The SVG has numbers converted to paths/outlines (not editable text).")
    print("  - Numbers include extra characters (e.g., '+2', '2%', '2.0').")
    print("If you want, I can adapt the matcher for patterns like '+2' or '(-2)'.")
else:
    output_svg.parent.mkdir(parents=True, exist_ok=True)
    tree.write(output_svg, encoding="utf-8", xml_declaration=True)
    print(f"✅ Updated {changed} number(s). Saved to:\n{output_svg}")